In [19]:
cd /home/ubuntu/Documents/jiarui/code/pFedDef

/home/ubuntu/Documents/jiarui/code/pFedDef


In [20]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *
from transfer_attacks.TA_utils import *
from transfer_attacks.Boundary_Transferer import *

In [21]:
# Generating Empty Aggregator to be loaded 

setting = 'FedAvg'

if setting == 'FedEM':
    nL = 3
else:
    nL = 1

# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = setting
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= nL
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/dummy/'
args_.validation = False
args_.aggregation_op = None

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_, num_user=40)

==> Clients initialization..
===> Building data iterators..


100%|██████████| 40/40 [00:00<00:00, 181.73it/s]


===> Initializing clients..


100%|██████████| 40/40 [00:06<00:00,  6.33it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.299 | Train Acc: 10.514% |Test Loss: 2.298 | Test Acc: 11.018% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [22]:
# Compiling Dataset from Clients
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].test_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
try:
    data_y = torch.stack(data_y)        
except:
    data_y = torch.FloatTensor(data_y) 

    
# mask_cat = data_y==3
# mask_dog = data_y==5
# data_x = torch.cat(
#     (data_x[mask_cat], data_x[mask_dog]),
#     0
# )
# data_y = torch.cat(
#     (data_y[mask_cat], data_y[mask_dog]),
#     0
# )
    
    
dataloader = Custom_Dataloader(data_x, data_y)

In [23]:
# Import Model Weights
num_models = 40

root_path = "/home/ubuntu/Documents/jiarui/experiments/FAT_progressive/FedAvg_adv_progressive/weights/avg_w"

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

if setting == 'local':

#     args_.save_path = 'weights/final/femnist/fig1_take3/local_benign/'
    args_.save_path ='weights/final/femnist/fig1_take3/local_adv/'
    aggregator.load_state(args_.save_path)
    
    model_weights = []
#     weights = np.load("weights/final/femnist/fig1_take3/local_benign/train_client_weights.npy")
    weights = np.load('weights/final/femnist/fig1_take3/local_adv/train_client_weights.npy')
    
    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for i in range(num_models):
        new_model = copy.deepcopy(aggregator.clients[i].learners_ensemble.learners[0].model)
        new_model.eval()
        models_test += [new_model]

elif setting == 'FedAvg':
    
#     args_.save_path = 'weights/final/femnist/fig1_take3/fedavg_benign/'
    args_.save_path = root_path
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

#     weights = np.load("weights/final/femnist/fig1_take3/fedavg_benign/train_client_weights.npy")
    weights = np.load(f'{root_path}/train_client_weights.npy')
    
    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0[0]*weights_h[0][key] 
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

elif setting == 'FedEM':
    
#     args_.save_path = 'weights/final/femnist/fig1_take3/fedem_benign/'
#     args_.save_path = 'weights/final/femnist/fig1_take3/fedem_adv/'
    args_.save_path = root_path
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

#     weights = np.load("weights/final/femnist/fig1_take3/fedem_benign/train_client_weights.npy")
#     weights = np.load("weights/final/femnist/fig1_take3/fedem_adv/train_client_weights.npy")
    weights = np.load(f"{root_path}/train_client_weights.npy")

    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]


    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0,w1,w2) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
#             print(key)
#             print(weights_h[0][key])
#             print(weights_h[1][key])
#             print(weights_h[2][key])
            new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

In [24]:
# Here we will make a dictionary that will hold results
logs_adv = []

for i in range(num_models):
    adv_dict = {}
    adv_dict['orig_acc_transfers'] = None
    adv_dict['orig_similarities'] = None
    adv_dict['adv_acc_transfers'] = None
    adv_dict['adv_similarities_target'] = None
    adv_dict['adv_similarities_untarget'] = None
    adv_dict['adv_target'] = None
    adv_dict['adv_miss'] = None
    adv_dict['metric_alignment'] = None
    adv_dict['ib_distance_legit'] = None
    adv_dict['ib_distance_adv'] = None

    logs_adv += [adv_dict]

In [25]:
# Perform transfer attack from one client to another and record stats

# Run Measurements for both targetted and untargeted analysis
new_num_models = len(models_test)
victim_idxs = range(new_num_models)
custom_batch_size = 500
eps = 4.5


for adv_idx in victim_idxs:
    print("\t Adv idx:", adv_idx)
    
    dataloader = load_client_data(
        clients = clients, 
        c_id = adv_idx, 
        mode = 'test',
        switch = False
    ) # or test/train
    
    batch_size = min(custom_batch_size, dataloader.y_data.shape[0])
    
    t1 = Transferer(models_list=models_test, dataloader=dataloader)
    t1.generate_victims(victim_idxs)
    
    # Perform Attacks Targeted
    t1.atk_params = PGD_Params()
    t1.atk_params.set_params(
        batch_size=batch_size,
        iteration = 10,
        target = 3, 
        x_val_min = torch.min(data_x), 
        x_val_max = torch.max(data_x),
        step_size = 0.01, 
        step_norm = "inf", 
        eps = eps, 
        eps_norm = 2
    )
    
    
    
    t1.generate_advNN(adv_idx)
    t1.generate_xadv(atk_type = "pgd")
    t1.send_to_victims(victim_idxs)

    # Log Performance
    logs_adv[adv_idx]['orig_acc_transfers'] = copy.deepcopy(t1.orig_acc_transfers)
    logs_adv[adv_idx]['orig_similarities'] = copy.deepcopy(t1.orig_similarities)
    logs_adv[adv_idx]['adv_acc_transfers'] = copy.deepcopy(t1.adv_acc_transfers)
    logs_adv[adv_idx]['adv_similarities_target'] = copy.deepcopy(t1.adv_similarities)        
    logs_adv[adv_idx]['adv_target'] = copy.deepcopy(t1.adv_target_hit)

#     Miss attack Untargeted
    
    t1.atk_params.set_params(
        batch_size=batch_size, 
        iteration = 10,
        target = -1, 
        x_val_min = torch.min(data_x), 
        x_val_max = torch.max(data_x),
        step_size = 0.01, 
        step_norm = "inf", 
        eps = eps, 
        eps_norm = 2
    )
    t1.generate_xadv(atk_type = "pgd")
    t1.send_to_victims(victim_idxs)
    logs_adv[adv_idx]['adv_miss'] = copy.deepcopy(t1.adv_acc_transfers)
    logs_adv[adv_idx]['adv_similarities_untarget'] = copy.deepcopy(t1.adv_similarities)

	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7
	 Adv idx: 8
	 Adv idx: 9
	 Adv idx: 10
	 Adv idx: 11
	 Adv idx: 12
	 Adv idx: 13
	 Adv idx: 14
	 Adv idx: 15
	 Adv idx: 16
	 Adv idx: 17
	 Adv idx: 18
	 Adv idx: 19
	 Adv idx: 20
	 Adv idx: 21
	 Adv idx: 22
	 Adv idx: 23
	 Adv idx: 24
	 Adv idx: 25
	 Adv idx: 26
	 Adv idx: 27
	 Adv idx: 28
	 Adv idx: 29
	 Adv idx: 30
	 Adv idx: 31
	 Adv idx: 32
	 Adv idx: 33
	 Adv idx: 34
	 Adv idx: 35
	 Adv idx: 36
	 Adv idx: 37
	 Adv idx: 38
	 Adv idx: 39


In [26]:
# Aggregate Results Across clients 
metrics = ['orig_acc_transfers','orig_similarities','adv_acc_transfers','adv_similarities_target',
           'adv_similarities_untarget','adv_target','adv_miss'] #,'metric_alignment']

orig_acc = np.zeros([len(victim_idxs),len(victim_idxs)]) 
orig_sim = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_acc = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_sim_target = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_sim_untarget = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_target = np.zeros([len(victim_idxs),len(victim_idxs)])
adv_miss = np.zeros([len(victim_idxs),len(victim_idxs)]) 

for adv_idx in range(len(victim_idxs)):
    for victim in range(len(victim_idxs)):
        orig_acc[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[0]][victim_idxs[victim]].data.tolist()
        orig_sim[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[1]][victim_idxs[victim]].data.tolist()
        adv_acc[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[2]][victim_idxs[victim]].data.tolist()
        adv_sim_target[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[3]][victim_idxs[victim]].data.tolist()
        adv_sim_untarget[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[4]][victim_idxs[victim]].data.tolist()
        adv_target[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[5]][victim_idxs[victim]].data.tolist()
        adv_miss[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[6]][victim_idxs[victim]].data.tolist()

In [27]:
print(np.sum(orig_acc) / (orig_acc.shape[0] * orig_acc.shape[1]))
print(np.sum(adv_acc) / (adv_acc.shape[0] * adv_acc.shape[1]))

0.8255093082785606
0.540223478525877


In [9]:
# Check Results
print(adv_target[:10,:10])

[[0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29]
 [0.30 0.30 0.30 0.30 0.30 0.30 0.30 0.30 0.30 0.30]
 [0.32 0.32 0.32 0.32 0.32 0.32 0.32 0.32 0.32 0.32]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
 [0.30 0.30 0.30 0.30 0.30 0.30 0.30 0.30 0.30 0.30]
 [0.30 0.30 0.30 0.30 0.30 0.30 0.30 0.30 0.30 0.30]
 [0.41 0.41 0.41 0.41 0.41 0.41 0.41 0.41 0.41 0.41]
 [0.24 0.24 0.24 0.24 0.24 0.24 0.24 0.24 0.24 0.24]
 [0.32 0.32 0.32 0.32 0.32 0.32 0.32 0.32 0.32 0.32]
 [0.30 0.30 0.30 0.30 0.30 0.30 0.30 0.30 0.30 0.30]]


In [17]:
orig_acc[:10, :10]

array([[0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90],
       [0.83, 0.83, 0.83, 0.83, 0.83, 0.83, 0.83, 0.83, 0.83, 0.83],
       [0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86],
       [1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00],
       [0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81],
       [0.84, 0.84, 0.84, 0.84, 0.84, 0.84, 0.84, 0.84, 0.84, 0.84],
       [0.82, 0.82, 0.82, 0.82, 0.82, 0.82, 0.82, 0.82, 0.82, 0.82],
       [0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81],
       [0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86],
       [0.83, 0.83, 0.83, 0.83, 0.83, 0.83, 0.83, 0.83, 0.83, 0.83]])

In [11]:
print(adv_acc[:10, :10])
print(np.sum(adv_acc) / (adv_acc.shape[0] * adv_acc.shape[1]))

[[0.61 0.61 0.61 0.61 0.61 0.61 0.61 0.61 0.61 0.61]
 [0.58 0.58 0.58 0.58 0.58 0.58 0.58 0.58 0.58 0.58]
 [0.60 0.60 0.60 0.60 0.60 0.60 0.60 0.60 0.60 0.60]
 [1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]
 [0.58 0.58 0.58 0.58 0.58 0.58 0.58 0.58 0.58 0.58]
 [0.63 0.63 0.63 0.63 0.63 0.63 0.63 0.63 0.63 0.63]
 [0.48 0.48 0.48 0.48 0.48 0.48 0.48 0.48 0.48 0.48]
 [0.64 0.64 0.64 0.64 0.64 0.64 0.64 0.64 0.64 0.64]
 [0.63 0.63 0.63 0.63 0.63 0.63 0.63 0.63 0.63 0.63]
 [0.58 0.58 0.58 0.58 0.58 0.58 0.58 0.58 0.58 0.58]]
0.6197192803025245


In [10]:
# np.save("/home/ubuntu/Documents/jiarui/experiments/pFedDef/weights/cifar10/FedAvg_all_label_switch/pfeddef/acc.npy", orig_acc)
np.save("/home/ubuntu/Documents/jiarui/experiments/FedAvg_adv/rep_atk/rep_scale0/eval/all_adv_acc.npy", adv_acc)
# np.load("/home/ubuntu/Documents/jiarui/experiments/FedAvg/cifar10/boosting80/eval/boosting80_all_acc.npy")[:10,:10]


In [63]:
print("\nfedavg")
print(np.load("/home/ubuntu/Documents/jiarui/experiments/pFedDef/eval/fedavg_all_acc.npy")[:10,:10])
print(np.load("/home/ubuntu/Documents/jiarui/experiments/pFedDef/eval/fedavg_pair_acc.npy")[:10,:10])

print("\nswitch pair - 20 clients")
print(np.load("/home/ubuntu/Documents/jiarui/experiments/pFedDef/eval/switchPair20_all_acc.npy")[:10,:10])
print(np.load("/home/ubuntu/Documents/jiarui/experiments/pFedDef/eval/switchPair20_pair_acc.npy")[:10,:10])

print("\nswitch pair - 40 clients")
print(np.load("/home/ubuntu/Documents/jiarui/experiments/FedAvg/cifar10/switchPair40/eval/switchPair40_all_acc.npy")[:10,:10])
print(np.load("/home/ubuntu/Documents/jiarui/experiments/FedAvg/cifar10/switchPair40/eval/switchPair40_pair_acc.npy")[:10,:10])

print("\nswitch all")
print(np.load("/home/ubuntu/Documents/jiarui/experiments/pFedDef/eval/allSwitch_all_acc.npy")[:10,:10])
print(np.load("/home/ubuntu/Documents/jiarui/experiments/pFedDef/eval/allSwitch_pair_acc.npy")[:10,:10])


fedavg
[[0.87 0.87 0.87 0.87 0.87 0.87 0.87 0.87 0.87 0.87]
 [0.81 0.81 0.81 0.81 0.81 0.81 0.81 0.81 0.81 0.81]
 [0.86 0.86 0.86 0.86 0.86 0.86 0.86 0.86 0.86 0.86]
 [1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]
 [0.80 0.80 0.80 0.80 0.80 0.80 0.80 0.80 0.80 0.80]
 [0.84 0.84 0.84 0.84 0.84 0.84 0.84 0.84 0.84 0.84]
 [0.81 0.81 0.81 0.81 0.81 0.81 0.81 0.81 0.81 0.81]
 [0.79 0.79 0.79 0.79 0.79 0.79 0.79 0.79 0.79 0.79]
 [0.86 0.86 0.86 0.86 0.86 0.86 0.86 0.86 0.86 0.86]
 [0.82 0.82 0.82 0.82 0.82 0.82 0.82 0.82 0.82 0.82]]
[[0.78 0.78 0.78 0.78 0.78 0.78 0.78 0.78 0.78 0.78]
 [0.74 0.74 0.74 0.74 0.74 0.74 0.74 0.74 0.74 0.74]
 [0.65 0.65 0.65 0.65 0.65 0.65 0.65 0.65 0.65 0.65]
 [1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]
 [0.70 0.70 0.70 0.70 0.70 0.70 0.70 0.70 0.70 0.70]
 [0.59 0.59 0.59 0.59 0.59 0.59 0.59 0.59 0.59 0.59]
 [0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68]
 [0.61 0.61 0.61 0.61 0.61 0.61 0.61 0.61 0.61 0.61]
 [0.62 0.62 0.62 0.62 0.62 0.62 0.62 

In [39]:
print("all acc")
# print("boost 20")
# print(np.load("/home/ubuntu/Documents/jiarui/experiments/FedAvg/cifar10/boosting20/eval/boosting20_all_acc.npy")[:10,:10])
print("boost 40")
print(np.load("/home/ubuntu/Documents/jiarui/experiments/FedAvg/cifar10/boosting40/eval/boosting40_all_acc.npy")[:10,:10])
print("boost 80")
print(np.load("/home/ubuntu/Documents/jiarui/experiments/FedAvg/cifar10/boosting80/eval/boosting80_all_acc.npy")[:10,:10])
print("boost 160")
print(np.load("/home/ubuntu/Documents/jiarui/experiments/FedAvg/cifar10/boosting160/eval/boosting160_all_acc.npy")[:10,:10])
print("pair acc")
# print("boost 20")
# print(np.load("/home/ubuntu/Documents/jiarui/experiments/FedAvg/cifar10/boosting20/eval/boosting20_pair_acc.npy")[:10,:10])
print("boost 40")
print(np.load("/home/ubuntu/Documents/jiarui/experiments/FedAvg/cifar10/boosting40/eval/boosting40_pair_acc.npy")[:10,:10])
print("boost 80")
print(np.load("/home/ubuntu/Documents/jiarui/experiments/FedAvg/cifar10/boosting80/eval/boosting80_pair_acc.npy")[:10,:10])
print("boost 160")
print(np.load("/home/ubuntu/Documents/jiarui/experiments/FedAvg/cifar10/boosting160/eval/boosting160_pair_acc.npy")[:10,:10])


all acc
boost 40
[[0.88 0.88 0.88 0.88 0.88 0.88 0.88 0.88 0.88 0.88]
 [0.79 0.79 0.79 0.79 0.79 0.79 0.79 0.79 0.79 0.79]
 [0.87 0.87 0.87 0.87 0.87 0.87 0.87 0.87 0.87 0.87]
 [1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]
 [0.83 0.83 0.83 0.83 0.83 0.83 0.83 0.83 0.83 0.83]
 [0.87 0.87 0.87 0.87 0.87 0.87 0.87 0.87 0.87 0.87]
 [0.80 0.80 0.80 0.80 0.80 0.80 0.80 0.80 0.80 0.80]
 [0.80 0.80 0.80 0.80 0.80 0.80 0.80 0.80 0.80 0.80]
 [0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85]
 [0.78 0.78 0.78 0.78 0.78 0.78 0.78 0.78 0.78 0.78]]
boost 80
[[0.87 0.87 0.87 0.87 0.87 0.87 0.87 0.87 0.87 0.87]
 [0.82 0.82 0.82 0.82 0.82 0.82 0.82 0.82 0.82 0.82]
 [0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85]
 [1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]
 [0.83 0.83 0.83 0.83 0.83 0.83 0.83 0.83 0.83 0.83]
 [0.84 0.84 0.84 0.84 0.84 0.84 0.84 0.84 0.84 0.84]
 [0.81 0.81 0.81 0.81 0.81 0.81 0.81 0.81 0.81 0.81]
 [0.80 0.80 0.80 0.80 0.80 0.80 0.80 0.80 0.80 0.80]
 [0.86 0.86 0.86 0.

In [7]:
mypath = "/home/ubuntu/Documents/jiarui/experiments/oriCode/testFAT/pfeddef/weights"
np.load(f"{mypath}/train_client_weights.npy")

array([[0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
 

In [5]:
start = 3201 # - 3200
end = 6401 # - 3200
for i in range(start,end):
    if i % 100 == 0:
        print(f"----- {i} -----")
    h_orig_category = torch.load(f"/home/ubuntu/Documents/jiarui/experiments/multi_atker/collapse/pred_results/h_orig_category{i}.pt")
    h_adv_category = torch.load(f"/home/ubuntu/Documents/jiarui/experiments/multi_atker/collapse/pred_results/h_adv_category{i}.pt")
    
    if torch.count_nonzero(h_orig_category) != 0:
        print(f"{i} - orig {h_orig_category}")
    if torch.count_nonzero(h_adv_category) != 0:
        print(f"{i} - adv {h_adv_category}")
#     print(len(h_orig_category))
#     print(len(h_adv_category))
#     print(h_orig_category)
#     print(h_adv_category)
    # break


----- 3300 -----
----- 3400 -----
3441 - orig tensor([6, 6, 6, 6], device='cuda:0')
3441 - adv tensor([6, 6, 6, 6], device='cuda:0')
3442 - orig tensor([6, 6, 6, 6], device='cuda:0')
3442 - adv tensor([6, 6, 6, 6], device='cuda:0')
3443 - orig tensor([6, 6, 6, 6], device='cuda:0')
3443 - adv tensor([6, 6, 6, 6], device='cuda:0')
3444 - orig tensor([6, 6, 6, 6], device='cuda:0')
3444 - adv tensor([6, 6, 6, 6], device='cuda:0')
3445 - orig tensor([6, 6, 6, 6], device='cuda:0')
3445 - adv tensor([6, 6, 6, 6], device='cuda:0')
3446 - orig tensor([6, 6, 6, 6], device='cuda:0')
3446 - adv tensor([6, 6, 6, 6], device='cuda:0')
3447 - orig tensor([6, 6, 6, 6], device='cuda:0')
3447 - adv tensor([6, 6, 6, 6], device='cuda:0')
3448 - orig tensor([6, 6, 6, 6], device='cuda:0')
3448 - adv tensor([6, 6, 6, 6], device='cuda:0')
3449 - orig tensor([6, 6, 6, 6], device='cuda:0')
3449 - adv tensor([6, 6, 6, 6], device='cuda:0')
3450 - orig tensor([6, 6, 6, 6], device='cuda:0')
3450 - adv tensor([6, 6, 